In [33]:
import pandas as pd 
import numpy as np
import random
random.seed(9001)
#pour avoir toujours les memes erreurs à chaque fois qu'on re exécute le projet.

In [34]:
#tab = pd.read_csv('ratings.csv')
tab = pd.read_csv('./ml-latest-small/ratings_new.csv')


In [35]:
useri,frequsers=np.unique(tab.userId,return_counts=True)#useri les id des users, frequsers les freq de chaque user
itemi,freqitems=np.unique(tab.movieId,return_counts=True)#itemi les id des item, freqitem les freq de chaque item
n_users=len(useri)
n_items=len(itemi)
print("le nombre des utilisateurs est :"+ str(n_users) + " Et le nombre des items est: "+ str(n_items))

le nombre des utilisateurs est :671 Et le nombre des items est: 9066


In [36]:
data_matrix = np.zeros((n_users, n_items))
for line in tab[["userIdnew","movieIdnew","rating"]].itertuples():#parcourire la ligne col par col
    data_matrix[line[1]-1, line[2]-1] = line[3] 

In [37]:
#functions to pass from old to new Id:
def change_item_to_newId(id):
    if id ==0: return (0)
    else: return tab.movieIdnew[tab.movieId==id].iloc[0]
def change_item_to_Id(id):
    return tab.movieId[tab.movieIdnew==id].iloc[0]

# 2.3 ALS : Alternating Least Squares

In [38]:
# Index matrix for training data
I = data_matrix.copy()
I[I > 0] = 1
I[I == 0] = 0



In [39]:
#ps: on n'a pas commenté car c'est presque le mm principe que le modele SGD!
lmbda = 0.1 
k = 20 
n_epochs = 2 # Nombre d'étapes
m, n = data_matrix.shape # Number of users and items
P = 3 * np.random.rand(k,m) # Latent user feature matrix
Q = 3 * np.random.rand(k,n) # Latent item feature matrix
Q[0,:] = data_matrix[data_matrix != 0].mean(axis=0) # Avg. rating for each movie
E = np.eye(k) # (k x k)-dimensional idendity matrix

In [40]:
def rmse2(I,R,Q,P):
    return np.sqrt(np.sum((I * (R - prediction(P,Q)))**2)/len(R[R > 0]))
# La fonction prediction permet de prédire les ratings inconnus en multipliant les matrices P et la transposée de Q
def prediction(P,Q):
    return np.dot(P.T,Q)

In [41]:
train_errors = []
test_errors = []

# Repeat until convergence
for epoch in range(n_epochs):
    # Fix Q and estimate P
    for i, Ii in enumerate(I):
        nui = np.count_nonzero(Ii) # Number of items user i has rated
        if (nui == 0): nui = 1 # Be aware of zero counts!
    
        # Least squares solution
        Ai = np.dot(Q, np.dot(np.diag(Ii), Q.T)) + lmbda * nui * E
        Vi = np.dot(Q, np.dot(np.diag(Ii), data_matrix[i].T))
        P[:,i] = np.linalg.solve(Ai,Vi)
        
    # Fix P and estimate Q
    for j, Ij in enumerate(I.T):
        nmj = np.count_nonzero(Ij) # Number of users that rated item j
        if (nmj == 0): nmj = 1 # Be aware of zero counts!
        
        # Least squares solution
        Aj = np.dot(P, np.dot(np.diag(Ij), P.T)) + lmbda * nmj * E
        Vj = np.dot(P, np.dot(np.diag(Ij), data_matrix[:,j]))
        Q[:,j] = np.linalg.solve(Aj,Vj)
    
    train_rmse = rmse2(I,data_matrix,Q,P)
    train_errors.append(train_rmse)
    
    
    print "[Epoch %d/%d] train error: %f" \
    %(epoch+1, n_epochs, train_rmse)
    
print "Algorithm converged"

[Epoch 1/2] train error: 1.044221
[Epoch 2/2] train error: 0.793219
Algorithm converged


In [42]:
model_matrix=prediction(P,Q)
model_df=pd.DataFrame(model_matrix)

In [44]:
model_df#matrice (user,item)
#model_df.to_csv('als_model.csv')

,0,1,2,3,4,5,6,7,8,9,...,9056,9057,9058,9059,9060,9061,9062,9063,9064,9065
0,2.130215,2.227043,2.176165,1.535699,1.788820,2.194948,2.457750,2.329552,2.251277,2.497897,...,1.312637,0.255754,1.333493,0.432518,0.648777,2.530709,2.000239,0.266939,0.160164,2.625274
1,3.341446,2.806277,2.761646,2.283285,2.415235,3.136351,2.769467,2.759379,1.914126,3.026145,...,2.078068,0.289808,2.127988,0.754753,1.132130,3.471309,3.191981,1.083501,0.650100,4.156136
2,3.155881,2.851886,2.556706,2.138765,2.789464,3.140729,2.884998,2.694170,2.649259,2.756431,...,1.543074,0.259683,2.210477,0.691509,1.037264,3.245351,3.315716,1.982852,1.189711,3.086147
3,4.258256,3.558652,3.434244,2.516338,3.582201,4.115987,3.685628,3.516517,3.051078,3.616054,...,2.046890,0.354136,2.958144,1.004141,1.506211,4.133349,4.437216,2.425517,1.455310,4.093781
4,3.781559,3.524377,3.321539,2.402033,3.176307,3.401772,3.518006,3.422324,2.814749,3.358846,...,1.926588,0.294023,2.330920,0.852946,1.279418,4.120618,3.496380,1.990597,1.194358,3.853176
5,3.111447,2.350872,2.583490,2.044328,2.319145,2.879564,2.802987,2.394540,2.138468,2.543183,...,1.986657,0.087854,2.212003,0.691092,1.036638,2.925421,3.318004,1.168457,0.701074,3.973315
6,3.546858,2.780076,2.642226,2.053873,2.665340,3.379519,2.734299,2.360386,1.945793,2.790052,...,1.679287,0.226839,2.418839,0.859329,1.288994,3.189017,3.628259,2.031686,1.219012,3.358574
7,3.419823,3.028269,2.679024,2.032303,2.769575,3.576390,2.734180,2.440309,2.535374,3.063372,...,1.680921,0.315941,2.608691,0.845535,1.268302,3.191557,3.913037,2.011054,1.206632,3.361842
8,3.519640,2.593809,2.327978,1.892562,2.528306,3.394759,2.458466,1.707811,1.971439,2.806698,...,1.866828,0.276925,2.902464,0.863149,1.294723,3.227092,4.353696,1.666740,1.000044,3.733657
9,3.451707,2.929807,2.557651,1.726553,2.570702,3.187641,3.074975,2.155611,2.482879,3.014653,...,1.896827,0.387625,2.364899,0.848898,1.273346,3.674637,3.547349,0.978377,0.587026,3.793654


In [45]:
def als_recom_it_for_user(id):#idx<-id, row<-ds.description[ds.id==id]
    id=id-1
    similar_indices = model_matrix[id].argsort()[:-20:-1]
    #model_matrix[id] les rate pour un utilisateur 
    #.argsort()[:-20:-1] ordonnée et on prend les 20 les plus grande
    similar_items = [(model_matrix[id][i], change_item_to_Id(i+1)) for i in similar_indices]
    #ps: i+1 pour éliminer l'effet du faite qu'on commance avec un zéro notre liste ditems!
    return similar_items[:]

In [46]:
als_recom_it_for_user(10)

[(4.7533110113955068, 67504),
 (4.7533110113955068, 83359),
 (4.7533110113955068, 83411),
 (4.7533110113955068, 83318),
 (4.5833303612090779, 4427),
 (4.5485859949516652, 213),
 (4.5434383847377067, 6669),
 (4.5385107387012109, 32525),
 (4.5261410576591707, 55069),
 (4.5200273229600407, 47728),
 (4.5200273229600407, 7208),
 (4.5200273229600407, 25852),
 (4.5200273229600407, 8121),
 (4.4725018491403299, 1611),
 (4.453466709160427, 90428),
 (4.4506925756033677, 994),
 (4.4440964729718972, 80),
 (4.4387307566757492, 26524),
 (4.4282222720534214, 4612)]

In [47]:
# ps: for the first user 1, the first recommandation is for the item 184 with a 3.42 as rate let's verify this:
#to verify we have to -=1!
model_matrix[10-1][change_item_to_newId(59684)-1]

4.1360250928914581

In [48]:
#get simialrity users:
#let's compute the similarty between users:
from sklearn.metrics.pairwise import linear_kernel
p_matrix=pd.DataFrame(P).corr()

In [49]:
def cf_similaire_user(id):#idx<-id, row<-ds.description[ds.id==id]
    id=id-1
    similar_indices = p_matrix[id].argsort()[:-20:-1]
    similar_items = [(p_matrix[id][i], change_item_to_Id(i+1)) for i in similar_indices]
    return similar_items[1:]

In [50]:
cf_similaire_user(10)#les utilisateurs les plus proches de l'utilisateur1 

[(0.83271120196398163, 587),
 (0.83045748271218356, 568),
 (0.8207446563724522, 137),
 (0.81854685843876029, 365),
 (0.80734750990521631, 281),
 (0.7955015363365715, 433),
 (0.78809664725811313, 742),
 (0.78722928002257697, 60),
 (0.78298247032844526, 550),
 (0.78201875917818409, 132),
 (0.77774256375237694, 305),
 (0.77397437555303461, 691),
 (0.76018978273768456, 354),
 (0.75808952039188693, 39),
 (0.75694645872370014, 333),
 (0.75669657493971221, 377),
 (0.75587410471296423, 512),
 (0.75060502596941392, 452)]

In [51]:
#let's verify this result:
p_matrix[10-1][change_item_to_newId(329)-1]#we have to do -=1 for the same reason

0.63141962488850611

In [52]:
#Let's do the same for items:
from sklearn.metrics.pairwise import linear_kernel
q_matrix=pd.DataFrame(Q).corr()

In [53]:
def cf_similaire_item(id,n=20):#idx<-id, row<-ds.description[ds.id==id]
    id=change_item_to_newId(id)-1
    similar_indices = q_matrix[id].argsort()[:-n:-1]
    similar_items = [(q_matrix[id][i], change_item_to_Id(i+1)) for i in similar_indices]
    return similar_items[1:]

In [313]:
cf_similaire_item(1029,20)

[(0.9848840772343318, 596),
 (0.98473741231587475, 135137),
 (0.98238084575734641, 1130),
 (0.98228048466534534, 594),
 (0.9811515773277133, 3897),
 (0.98112934297573706, 2804),
 (0.98107422969374891, 1674),
 (0.98089422842828411, 3252),
 (0.98032462112294283, 1032),
 (0.98024307191820714, 2183),
 (0.97993144137321064, 954),
 (0.97990415799550667, 1086),
 (0.97988711650489102, 357),
 (0.97972014065586077, 39),
 (0.97882431172199891, 4034),
 (0.97873265672299281, 78034),
 (0.97856952952940468, 1304),
 (0.97846266529755288, 2090)]

In [54]:
#let's verify this result:
q_matrix[change_item_to_newId(1029)-1][change_item_to_newId(596-1)]#we have to do -=1 for the same reason

0.97927783730026041

In [55]:
#here is a function witch takes a userId and an itemId and return the rate predicted:
def als_rate_for_user_item(idu,idi):
    idu=idu-1
    idi=change_item_to_newId(idi)-1
    return model_matrix[idu,idi]

In [56]:
als_rate_for_user_item(10,1029)

3.4044619367440272

In [57]:
#let's verify:
model_matrix[(10-1),change_item_to_newId(1029)-1]

3.4044619367440272

In [58]:
#let's try to find the correlation between two users:
def cf_cor_user_user(id1,id2):#idx<-id, row<-ds.description[ds.id==id]
    id1=id1-1
    id2=id2-1
    return p_matrix[id1][id2]

In [59]:
cf_cor_user_user(3,4)

0.74603207285812168

In [60]:
#let's verify:
p_matrix[(3-1)][(4-1)]

0.74603207285812168

In [61]:
#let's try to find the correlation between two items:
def cf_cor_item_item(id1,id2):#idx<-id, row<-ds.description[ds.id==id]
    id1=change_item_to_newId(id1)-1
    id2=change_item_to_newId(id2)-1
    return q_matrix[id1][id2]

In [62]:
cf_cor_item_item(3,4)

0.77700102476964183

In [63]:
#let's verify:
q_matrix[change_item_to_newId(3-1)][change_item_to_newId(4-1)]

0.77700102476964183

In [64]:
#let's try to find the correlation between a user and an item:
import scipy.stats as sp
def cf_cor_user_item(idu,idi):#idx<-id, row<-ds.description[ds.id==id]
    idu=idu-1
    idi=change_item_to_newId(idi)-1
    return sp.pearsonr(P.T[idu],Q.T[idi])[0]

In [65]:
cf_cor_user_item(3,4)

0.77341226960571752

# 2. Content Based:

In [66]:
carac = pd.read_csv('./ml-latest-small/movies.csv')


In [78]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

carac['Sgenres']=np.NaN
for index,row in carac.iterrows() :
    lgenre=(carac.ix[index,'genres']).split('|')
    carac.ix[index,'Sgenres']=' ' .join(lgenre) 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(carac['Sgenres'].tolist())
Xarray=X.toarray()
cosine_similarities = cosine_similarity(Xarray, Xarray)

In [79]:
def cb_similaire_item(id,n=20):#idx<-id, row<-ds.description[ds.id==id]
    id=id-1
    similar_indices = cosine_similarities[id].argsort()[:-n:-1]
    similar_items = [(cosine_similarities[id][i], carac['movieId'][i]) for i in similar_indices]
    return similar_items[1:]

In [81]:
#cb_similaire_item(1,20)

In [71]:
#let's verify: 
print ("items similar to this item which has these genres: '" + carac.genres[carac.movieId==1][0]+"'" )
print ("has as genres:'"+carac.genres[carac.movieId==3114][2506]+"', '"+ carac.genres[carac.movieId==114552][8676]+"'")
print ("after we have : '"+ carac.genres[carac.movieId==108932][8517]+"'")

items similar to this item which has these genres: 'Adventure|Animation|Children|Comedy|Fantasy'
has as genres:'Adventure|Animation|Children|Comedy|Fantasy', 'Adventure|Animation|Children|Comedy|Fantasy'
after we have : 'Action|Adventure|Animation|Children|Comedy|Fantasy'


ps cette base de données contient plus de movies que la base de données rating mais c'est pas grave ;)

In [318]:
#cf_similaire_item(1029,n_items)

[(0.9848840772343318, 596),
 (0.98473741231587475, 135137),
 (0.98238084575734641, 1130),
 (0.98228048466534534, 594),
 (0.9811515773277133, 3897),
 (0.98112934297573706, 2804),
 (0.98107422969374891, 1674),
 (0.98089422842828411, 3252),
 (0.98032462112294283, 1032),
 (0.98024307191820714, 2183),
 (0.97993144137321064, 954),
 (0.97990415799550667, 1086),
 (0.97988711650489102, 357),
 (0.97972014065586077, 39),
 (0.97882431172199891, 4034),
 (0.97873265672299281, 78034),
 (0.97856952952940468, 1304),
 (0.97846266529755288, 2090),
 (0.97825061885279874, 1307),
 (0.97821021010853038, 3088),
 (0.97797664930784312, 1012),
 (0.97788367586162905, 661),
 (0.97772185845243686, 1953),
 (0.97767628607095047, 2735),
 (0.97754291557320161, 1022),
 (0.9770731730730805, 3035),
 (0.97702003312009522, 949),
 (0.97656895864368487, 1011),
 (0.97649584871035744, 1250),
 (0.97629468458461333, 953),
 (0.97623034616741311, 904),
 (0.97623023042977619, 2085),
 (0.97620021465503093, 4022),
 (0.9761990986351059

In [324]:
#cb_similaire_item(1029,n_items)

[(1.0, 95182),
 (1.0, 61160),
 (1.0, 1274),
 (0.92978345833954179, 95165),
 (0.90069704540382578, 71057),
 (0.90069704540382578, 71318),
 (0.70499728408075024, 2495),
 (0.70499728408075024, 105769),
 (0.70499728408075024, 5072),
 (0.70499728408075024, 741),
 (0.58781147919914556, 149406),
 (0.58781147919914556, 146309),
 (0.58781147919914556, 112175),
 (0.58781147919914556, 95780),
 (0.58781147919914556, 96007),
 (0.58781147919914556, 96004),
 (0.58781147919914556, 95965),
 (0.58781147919914556, 95782),
 (0.58781147919914556, 95963),
 (0.58781147919914556, 103210),
 (0.58781147919914556, 95499),
 (0.58781147919914556, 64695),
 (0.58781147919914556, 95473),
 (0.58781147919914556, 95475),
 (0.56163276327055767, 136449),
 (0.56163276327055767, 98124),
 (0.56163276327055767, 104419),
 (0.56163276327055767, 98607),
 (0.56163276327055767, 70533),
 (0.54127246734399825, 67408),
 (0.51671125974614329, 6283),
 (0.46667708641596767, 95147),
 (0.46667708641596767, 95167),
 (0.46667708641596767, 1

In [82]:
l=n_items
def hybrid(i):#i est l'indice du film pour lequel on veut chercher les items similaires
    cb=cb_similaire_item(i,l)#content based filtering based on genre only !!
    cf=cf_similaire_item(i,l)#collaborative filtering
    
    df1=pd.DataFrame(cb,columns=['score_cb','movieId'])
    df2=pd.DataFrame(cf,columns=['score_cf','movieId'])
    
    df_all=df1.join(df2.set_index('movieId'), on='movieId')
    df_all["score_all"]=df_all["score_cb"]+df_all["score_cf"]
    df_all=df_all.sort_values(by='score_all',ascending=False)
    
    return df_all

In [84]:
(hybrid(1)).head(4)

,score_cb,movieId,score_cf,score_all
5,1.0,4886,0.983677,1.983677
4,1.0,3114,0.982578,1.982578
0,1.0,2294,0.962589,1.962589
8,1.0,136016,0.938667,1.938667
